In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import duckdb

In [2]:
# Initialize DuckDB connection
con = duckdb.connect()
con.execute("SET s3_region='us-east-1'")

S3_BASE = "s3://jax-envision-public-data/study_1001/2025v3.3/tabular"

In [3]:
# Dose mappings for each replicate
DOSE_MAPPING_REP1 = {
    4917: '5 mg/kg',  4918: 'Vehicle',  4919: '25 mg/kg',
    4920: '25 mg/kg', 4921: '5 mg/kg',  4922: 'Vehicle',
    4923: 'Vehicle',  4924: '25 mg/kg', 4925: '5 mg/kg'
}

DOSE_MAPPING_REP2 = {
    4926: '25 mg/kg', 4927: '5 mg/kg',  4928: 'Vehicle',
    4929: 'Vehicle',  4930: '25 mg/kg', 4931: '5 mg/kg',
    4932: '5 mg/kg',  4933: '25 mg/kg', 4934: 'Vehicle'
}

In [4]:
# Data is stored in UTC, so we need UTC injection times
INJECTION_EVENTS = [
    {
        'name': 'Replicate 1, Dose 1',
        'short_name': 'Rep1_Dose1',
        'injection_time_utc': pd.Timestamp('2025-01-14 11:00:00'),  # 6 AM EST = 11 AM UTC
        'dates_to_load': ['2025-01-14'],  # 11 AM UTC, so same day is fine
        'cages': list(DOSE_MAPPING_REP1.keys()),
        'dose_mapping': DOSE_MAPPING_REP1,
        'time_of_day_est': '6:00 AM EST (lights on)',
        'circadian': 'Light phase start'
    },
    {
        'name': 'Replicate 1, Dose 2',
        'short_name': 'Rep1_Dose2',
        'injection_time_utc': pd.Timestamp('2025-01-17 22:00:00'),  # 5 PM EST = 10 PM UTC
        'dates_to_load': ['2025-01-17', '2025-01-18'],  # 10 PM UTC, need next day for post-injection
        'cages': list(DOSE_MAPPING_REP1.keys()),
        'dose_mapping': DOSE_MAPPING_REP1,
        'time_of_day_est': '5:00 PM EST (lights off)',
        'circadian': 'Dark phase start'
    },
    {
        'name': 'Replicate 2, Dose 1',
        'short_name': 'Rep2_Dose1',
        'injection_time_utc': pd.Timestamp('2025-01-28 22:00:00'),  # 5 PM EST = 10 PM UTC
        'dates_to_load': ['2025-01-28', '2025-01-29'],  # 10 PM UTC, need next day for post-injection
        'cages': list(DOSE_MAPPING_REP2.keys()),
        'dose_mapping': DOSE_MAPPING_REP2,
        'time_of_day_est': '5:00 PM EST (lights off)',
        'circadian': 'Dark phase start'
    },
    {
        'name': 'Replicate 2, Dose 2',
        'short_name': 'Rep2_Dose2',
        'injection_time_utc': pd.Timestamp('2025-01-31 11:00:00'),  # 6 AM EST = 11 AM UTC
        'dates_to_load': ['2025-01-31'],  # 11 AM UTC, same day is fine
        'cages': list(DOSE_MAPPING_REP2.keys()),
        'dose_mapping': DOSE_MAPPING_REP2,
        'time_of_day_est': '6:00 AM EST (lights on)',
        'circadian': 'Light phase start'
    }
]

In [5]:
# Analysis parameters
HOURS_BEFORE = 2
HOURS_AFTER = 8

In [6]:
def load_cage_data(cage_id, date_str, dose_mapping, file_type='animal_activity_db'):
    """Load parquet data for a specific cage and date."""
    path = f"{S3_BASE}/cage_id={cage_id}/date={date_str}/{file_type}.parquet"
    query = f"SELECT * FROM read_parquet('{path}')"
    
    try:
        df = con.execute(query).fetchdf()
        df['cage_id'] = cage_id
        df['dose_group'] = dose_mapping[cage_id]
        return df
    except Exception as e:
        print(f"    ✗ Cage {cage_id}, Date {date_str}: {e}")
        return pd.DataFrame()

In [7]:
def load_injection_event_data(event):
    """Load all data for a single injection event, handling UTC time correctly."""
    print(f"\n{'='*60}")
    print(f"Loading: {event['name']}")
    print(f"  Injection time (UTC): {event['injection_time_utc']}")
    print(f"  Injection time (EST): {event['time_of_day_est']}")
    print(f"  Dates to load: {event['dates_to_load']}")
    print(f"{'='*60}")
    
    dfs = []
    
    # Load data for each date needed
    for date_str in event['dates_to_load']:
        print(f"\n  Loading date: {date_str}")
        for cage_id in event['cages']:
            df = load_cage_data(
                cage_id, 
                date_str, 
                event['dose_mapping']
            )
            if not df.empty:
                print(f"    ✓ Cage {cage_id} ({event['dose_mapping'][cage_id]}): {len(df):,} rows")
                dfs.append(df)
    
    if not dfs:
        print("  ⚠ No data loaded!")
        return pd.DataFrame()
    
    df_all = pd.concat(dfs, ignore_index=True)
    print(f"\n  Total rows loaded: {len(df_all):,}")
    
    # Filter to locomotion at 60s resolution
    df_loco = df_all[
        (df_all['name'] == 'animal_bouts.locomotion') & 
        (df_all['resolution'] == 60)
    ].copy()
    
    # Convert time to datetime (data is in UTC)
    df_loco['time'] = pd.to_datetime(df_loco['time'])
    
    # Compute minutes from injection using UTC injection time
    df_loco['minutes_from_injection'] = (
        df_loco['time'] - event['injection_time_utc']
    ).dt.total_seconds() / 60
    
    # Filter to analysis window
    df_loco = df_loco[
        (df_loco['minutes_from_injection'] >= -HOURS_BEFORE * 60) &
        (df_loco['minutes_from_injection'] <= HOURS_AFTER * 60)
    ]
    
    print(f"  Locomotion data after filtering: {len(df_loco):,} rows")
    print(f"  Time range: {df_loco['minutes_from_injection'].min():.0f} to {df_loco['minutes_from_injection'].max():.0f} minutes")
    print(f"  Animals per dose group:")
    for dose in ['Vehicle', '5 mg/kg', '25 mg/kg']:
        n = df_loco[df_loco['dose_group'] == dose]['animal_id'].nunique()
        print(f"    {dose}: {n} animals")
    
    return df_loco

In [8]:
def plot_locomotor_by_dose_on_ax(df, ax, title, show_legend=True):
    """Plot mean ± SEM for each dose group on a given axis."""
    
    colors = {
        'Vehicle': '#2ecc71',
        '5 mg/kg': '#3498db',
        '25 mg/kg': '#e74c3c'
    }
    
    dose_order = ['Vehicle', '5 mg/kg', '25 mg/kg']
    
    for dose in dose_order:
        dose_data = df[df['dose_group'] == dose].copy()
        
        if dose_data.empty:
            continue
            
        dose_data['time_bin'] = dose_data['minutes_from_injection'].round()
        
        grouped = dose_data.groupby('time_bin')['value'].agg(['mean', 'std', 'count'])
        grouped['sem'] = grouped['std'] / np.sqrt(grouped['count'])
        grouped = grouped.reset_index()
        
        ax.plot(grouped['time_bin'], grouped['mean'], 
                color=colors[dose], label=dose, linewidth=2)
        
        ax.fill_between(grouped['time_bin'],
                        grouped['mean'] - grouped['sem'],
                        grouped['mean'] + grouped['sem'],
                        color=colors[dose], alpha=0.2)
    
    ax.axvline(x=0, color='black', linestyle='--', linewidth=2)
    ax.set_xlabel('Minutes from Injection', fontsize=10)
    ax.set_ylabel('Locomotion (proportion of time)', fontsize=10)
    ax.set_title(title, fontsize=11, fontweight='bold')
    ax.grid(True, alpha=0.3)
    ax.set_ylim(-0.05, 1.0)
    
    if show_legend:
        ax.legend(loc='upper right', fontsize=9)


In [9]:
def plot_individual_traces_on_ax(df, ax, title, dose_group, color):
    """Plot individual animal traces for a dose group."""
    
    for animal_id in df[df['dose_group'] == dose_group]['animal_id'].unique():
        animal_data = df[
            (df['animal_id'] == animal_id) & 
            (df['dose_group'] == dose_group)
        ].sort_values('minutes_from_injection')
        
        ax.plot(animal_data['minutes_from_injection'], 
                animal_data['value'],
                color=color, alpha=0.3, linewidth=0.8)
    
    # Plot mean as thick black line
    dose_data = df[df['dose_group'] == dose_group].copy()
    if not dose_data.empty:
        dose_data['time_bin'] = dose_data['minutes_from_injection'].round()
        grouped = dose_data.groupby('time_bin')['value'].mean().reset_index()
        ax.plot(grouped['time_bin'], grouped['value'], 
                color='black', linewidth=2, label='Mean')
    
    ax.axvline(x=0, color='red', linestyle='--', linewidth=1.5)
    ax.set_xlabel('Minutes from Injection', fontsize=10)
    ax.set_ylabel('Locomotion', fontsize=10)
    
    n_animals = df[df['dose_group'] == dose_group]['animal_id'].nunique()
    ax.set_title(f"{title}\n{dose_group} (n={n_animals})", fontsize=10)
    ax.set_ylim(-0.05, 1.1)
    ax.grid(True, alpha=0.3)

In [10]:
print("=" * 70)
print("MORPHINE LOCOMOTOR ANALYSIS - ALL INJECTION EVENTS")
print("=" * 70)
print(f"\nAnalysis window: {HOURS_BEFORE} hours before to {HOURS_AFTER} hours after injection")
print("All times converted to UTC for data alignment")

# Load all data
all_event_data = {}

for event in INJECTION_EVENTS:
    df_loco = load_injection_event_data(event)
    all_event_data[event['short_name']] = {
        'data': df_loco,
        'event': event
    }

MORPHINE LOCOMOTOR ANALYSIS - ALL INJECTION EVENTS

Analysis window: 2 hours before to 8 hours after injection
All times converted to UTC for data alignment

Loading: Replicate 1, Dose 1
  Injection time (UTC): 2025-01-14 11:00:00
  Injection time (EST): 6:00 AM EST (lights on)
  Dates to load: ['2025-01-14']

  Loading date: 2025-01-14


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

    ✓ Cage 4917 (5 mg/kg): 913,545 rows


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

    ✓ Cage 4918 (Vehicle): 849,024 rows


...

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

    ✓ Cage 4924 (25 mg/kg): 810,030 rows


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

    ✓ Cage 4925 (5 mg/kg): 833,949 rows

  Total rows loaded: 15,155,400


KeyboardInterrupt: 

In [ ]:

print("\n" + "=" * 70)
print("GENERATING 2x2 SUMMARY FIGURE")
print("=" * 70)

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

plot_positions = [
    ('Rep1_Dose1', axes[0, 0]),
    ('Rep1_Dose2', axes[0, 1]),
    ('Rep2_Dose1', axes[1, 0]),
    ('Rep2_Dose2', axes[1, 1]),
]

for short_name, ax in plot_positions:
    event_info = all_event_data[short_name]
    df = event_info['data']
    event = event_info['event']
    
    title = f"{event['name']}\n{event['time_of_day_est']}"
    plot_locomotor_by_dose_on_ax(df, ax, title, show_legend=(short_name == 'Rep1_Dose1'))

plt.suptitle('Morphine Effects on Locomotor Activity Across All Injection Events', 
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('all_injection_events_summary.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Saved: all_injection_events_summary.png")

In [ ]:
print("\n" + "=" * 70)
print("GENERATING INDIVIDUAL TRACES FIGURE")
print("=" * 70)

fig, axes = plt.subplots(4, 3, figsize=(18, 20))

colors = {
    'Vehicle': '#2ecc71',
    '5 mg/kg': '#3498db',
    '25 mg/kg': '#e74c3c'
}

dose_order = ['Vehicle', '5 mg/kg', '25 mg/kg']

for row_idx, short_name in enumerate(['Rep1_Dose1', 'Rep1_Dose2', 'Rep2_Dose1', 'Rep2_Dose2']):
    event_info = all_event_data[short_name]
    df = event_info['data']
    event = event_info['event']
    
    for col_idx, dose in enumerate(dose_order):
        ax = axes[row_idx, col_idx]
        title = f"{event['name']}"
        plot_individual_traces_on_ax(df, ax, title, dose, colors[dose])

plt.suptitle('Individual Animal Traces Across All Injection Events', 
             fontsize=16, fontweight='bold', y=1.01)
plt.tight_layout()
plt.savefig('all_events_individual_traces.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Saved: all_events_individual_traces.png")

In [ ]:
print("\n" + "=" * 70)
print("GENERATING CIRCADIAN COMPARISON")
print("=" * 70)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 6 AM injections (light phase start)
ax_am = axes[0]
for short_name, color in [('Rep1_Dose1', '#e74c3c'), ('Rep2_Dose2', '#c0392b')]:
    event_info = all_event_data[short_name]
    df = event_info['data']
    
    dose_data = df[df['dose_group'] == '25 mg/kg'].copy()
    if not dose_data.empty:
        dose_data['time_bin'] = dose_data['minutes_from_injection'].round()
        grouped = dose_data.groupby('time_bin')['value'].agg(['mean', 'std', 'count'])
        grouped['sem'] = grouped['std'] / np.sqrt(grouped['count'])
        grouped = grouped.reset_index()
        
        label = 'Rep 1, Dose 1' if 'Rep1' in short_name else 'Rep 2, Dose 2'
        ax_am.plot(grouped['time_bin'], grouped['mean'], linewidth=2, label=label, color=color)
        ax_am.fill_between(grouped['time_bin'],
                           grouped['mean'] - grouped['sem'],
                           grouped['mean'] + grouped['sem'],
                           alpha=0.2, color=color)

ax_am.axvline(x=0, color='black', linestyle='--', linewidth=2)
ax_am.set_xlabel('Minutes from Injection', fontsize=12)
ax_am.set_ylabel('Locomotion (proportion of time)', fontsize=12)
ax_am.set_title('6:00 AM EST Injections (Light Phase Start)\n25 mg/kg Only', fontsize=14, fontweight='bold')
ax_am.legend(fontsize=11)
ax_am.grid(True, alpha=0.3)
ax_am.set_ylim(-0.05, 1.0)

# 5 PM injections (dark phase start)
ax_pm = axes[1]
for short_name, color in [('Rep1_Dose2', '#3498db'), ('Rep2_Dose1', '#2980b9')]:
    event_info = all_event_data[short_name]
    df = event_info['data']
    
    dose_data = df[df['dose_group'] == '25 mg/kg'].copy()
    if not dose_data.empty:
        dose_data['time_bin'] = dose_data['minutes_from_injection'].round()
        grouped = dose_data.groupby('time_bin')['value'].agg(['mean', 'std', 'count'])
        grouped['sem'] = grouped['std'] / np.sqrt(grouped['count'])
        grouped = grouped.reset_index()
        
        label = 'Rep 1, Dose 2' if 'Rep1' in short_name else 'Rep 2, Dose 1'
        ax_pm.plot(grouped['time_bin'], grouped['mean'], linewidth=2, label=label, color=color)
        ax_pm.fill_between(grouped['time_bin'],
                           grouped['mean'] - grouped['sem'],
                           grouped['mean'] + grouped['sem'],
                           alpha=0.2, color=color)

ax_pm.axvline(x=0, color='black', linestyle='--', linewidth=2)
ax_pm.set_xlabel('Minutes from Injection', fontsize=12)
ax_pm.set_ylabel('Locomotion (proportion of time)', fontsize=12)
ax_pm.set_title('5:00 PM EST Injections (Dark Phase Start)\n25 mg/kg Only', fontsize=14, fontweight='bold')
ax_pm.legend(fontsize=11)
ax_pm.grid(True, alpha=0.3)
ax_pm.set_ylim(-0.05, 1.0)

plt.suptitle('Circadian Comparison: Morning vs Evening Morphine Injection', 
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('circadian_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Saved: circadian_comparison.png")

In [ ]:
# =============================================================================
# PLOT 4: DOSE 1 vs DOSE 2 (Sensitization/Tolerance)
# =============================================================================

print("\n" + "=" * 70)
print("GENERATING SENSITIZATION/TOLERANCE COMPARISON")
print("=" * 70)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Replicate 1
ax_rep1 = axes[0]
for short_name, linestyle, color in [('Rep1_Dose1', '-', '#e74c3c'), ('Rep1_Dose2', '--', '#3498db')]:
    event_info = all_event_data[short_name]
    df = event_info['data']
    event = event_info['event']
    
    dose_data = df[df['dose_group'] == '25 mg/kg'].copy()
    if not dose_data.empty:
        dose_data['time_bin'] = dose_data['minutes_from_injection'].round()
        grouped = dose_data.groupby('time_bin')['value'].agg(['mean', 'std', 'count'])
        grouped['sem'] = grouped['std'] / np.sqrt(grouped['count'])
        grouped = grouped.reset_index()
        
        label = f"Dose 1 ({event['time_of_day_est'].split()[0]})" if 'Dose1' in short_name else f"Dose 2 ({event['time_of_day_est'].split()[0]})"
        ax_rep1.plot(grouped['time_bin'], grouped['mean'], 
                     linewidth=2, linestyle=linestyle, label=label, color=color)
        ax_rep1.fill_between(grouped['time_bin'],
                             grouped['mean'] - grouped['sem'],
                             grouped['mean'] + grouped['sem'],
                             alpha=0.15, color=color)

ax_rep1.axvline(x=0, color='black', linestyle='--', linewidth=2)
ax_rep1.set_xlabel('Minutes from Injection', fontsize=12)
ax_rep1.set_ylabel('Locomotion (proportion of time)', fontsize=12)
ax_rep1.set_title('Replicate 1: First vs Second Dose\n25 mg/kg Only', fontsize=14, fontweight='bold')
ax_rep1.legend(fontsize=11)
ax_rep1.grid(True, alpha=0.3)
ax_rep1.set_ylim(-0.05, 1.0)

# Replicate 2
ax_rep2 = axes[1]
for short_name, linestyle, color in [('Rep2_Dose1', '-', '#3498db'), ('Rep2_Dose2', '--', '#e74c3c')]:
    event_info = all_event_data[short_name]
    df = event_info['data']
    event = event_info['event']
    
    dose_data = df[df['dose_group'] == '25 mg/kg'].copy()
    if not dose_data.empty:
        dose_data['time_bin'] = dose_data['minutes_from_injection'].round()
        grouped = dose_data.groupby('time_bin')['value'].agg(['mean', 'std', 'count'])
        grouped['sem'] = grouped['std'] / np.sqrt(grouped['count'])
        grouped = grouped.reset_index()
        
        label = f"Dose 1 ({event['time_of_day_est'].split()[0]})" if 'Dose1' in short_name else f"Dose 2 ({event['time_of_day_est'].split()[0]})"
        ax_rep2.plot(grouped['time_bin'], grouped['mean'], 
                     linewidth=2, linestyle=linestyle, label=label, color=color)
        ax_rep2.fill_between(grouped['time_bin'],
                             grouped['mean'] - grouped['sem'],
                             grouped['mean'] + grouped['sem'],
                             alpha=0.15, color=color)

ax_rep2.axvline(x=0, color='black', linestyle='--', linewidth=2)
ax_rep2.set_xlabel('Minutes from Injection', fontsize=12)
ax_rep2.set_ylabel('Locomotion (proportion of time)', fontsize=12)
ax_rep2.set_title('Replicate 2: First vs Second Dose\n25 mg/kg Only', fontsize=14, fontweight='bold')
ax_rep2.legend(fontsize=11)
ax_rep2.grid(True, alpha=0.3)
ax_rep2.set_ylim(-0.05, 1.0)

plt.suptitle('Sensitization Check: Does Response Change with Repeated Dosing?', 
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('dose1_vs_dose2_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Saved: dose1_vs_dose2_comparison.png")

In [ ]:
# =============================================================================
# SUMMARY STATISTICS TABLE
# =============================================================================

print("\n" + "=" * 70)
print("SUMMARY STATISTICS")
print("=" * 70)

summary_stats = []

for short_name, event_info in all_event_data.items():
    df = event_info['data']
    event = event_info['event']
    
    for dose in ['Vehicle', '5 mg/kg', '25 mg/kg']:
        dose_data = df[df['dose_group'] == dose]
        
        if dose_data.empty:
            continue
        
        # Pre-injection baseline (last 30 min before)
        pre = dose_data[
            (dose_data['minutes_from_injection'] >= -30) & 
            (dose_data['minutes_from_injection'] < 0)
        ]['value']
        
        # Peak period (60-180 min post)
        peak = dose_data[
            (dose_data['minutes_from_injection'] >= 60) & 
            (dose_data['minutes_from_injection'] <= 180)
        ]['value']
        
        fold_change = peak.mean() / pre.mean() if len(pre) > 0 and pre.mean() > 0 else np.nan
        
        summary_stats.append({
            'Event': event['name'],
            'Time_EST': event['time_of_day_est'].split('(')[0].strip(),
            'Dose': dose,
            'N_animals': dose_data['animal_id'].nunique(),
            'Baseline_mean': round(pre.mean(), 4) if len(pre) > 0 else np.nan,
            'Peak_mean': round(peak.mean(), 4) if len(peak) > 0 else np.nan,
            'Fold_change': round(fold_change, 2) if not np.isnan(fold_change) else np.nan
        })

df_summary = pd.DataFrame(summary_stats)
print("\n")
print(df_summary.to_string(index=False))

df_summary.to_csv('all_events_summary_stats.csv', index=False)
print("\n✓ Saved: all_events_summary_stats.csv")

print("\n" + "=" * 70)
print("ANALYSIS COMPLETE")
print("=" * 70)